In [1]:
import pandas as pd
import numpy as np

WRKDIR = '/Users/mooreank/Desktop/Raph/Deconvolution/transcript_data/'

In [2]:
#load expression data
quants_df = pd.read_hdf('{}ppmi.v121018.vst.quants.hdf5'.format(WRKDIR))
#load the gencode transcript annotations data
gencode = pd.read_csv('{}gencode.v29.transcripts.txt.gz'.format(WRKDIR), sep='\t', comment='#')
#create a dictionary of gene IDs (current feature/column labels) to gene names map
id_dict = dict(zip(gencode['gene_id'], gencode['gene_name'])) 
#rename the features/columns using the dict (mapping)
quants_df.rename(columns=id_dict, inplace=True)

In [3]:
quants_df.shape

(4648, 58780)

In [3]:
#transpose dataframe
new_quants_df = quants_df.transpose()

##making subset  of 400 samples
#subset_df = quants_df.iloc[:500,0:20000]
#subset_df = new_quants_df.iloc[:,0:25]

##drop duplicate genes
#duplicates = set([x for x in new_subset_df.index if new_subset_df.index.count(x) > 1])
subset_df_nodrop = new_quants_df
#subset_df_drop = subset_df.drop_duplicates()
subset_df_drop = new_quants_df.drop_duplicates()


In [4]:
##check for duplicate genes
test = pd.DataFrame(subset_df_drop.duplicated())
check = test[test[0]==True]
duplicate_genes = check.index.tolist()
len(duplicate_genes)

0

In [5]:
subset_df_drop.head()

,3081_BL,3447_BL,3232_BL,3764_BL,3810_BL,3201_BL,3057_V06,4117_BL,3174_V04,3661_V04,...,41283_V02,50746_V02,41282_V02,41375_V02,41292_V04,41356_V02,41296_V02,41294_V04,41314_V02,42308_V02
ID,,,,,,,,,,,,,,,,,,,,,
DDX11L1,7.695558,7.868893,7.949052,7.743883,7.827080,7.685133,7.560175,7.560175,7.811213,7.560175,...,7.652496,7.845281,7.792933,7.666131,7.722640,7.757751,7.705990,7.802658,7.808271,7.933548
WASH7P,8.834862,8.428663,8.680132,8.583878,8.692863,8.771154,8.672400,8.432619,8.777378,8.901476,...,8.816151,8.824504,8.628513,8.363795,8.567642,8.971531,8.445468,8.450284,8.683383,8.758711
MIR6859-1,7.891189,7.827659,7.785145,7.859833,7.937102,7.839242,7.692647,7.717507,7.560175,7.801363,...,8.256613,7.962724,7.828844,7.877476,8.018015,8.024884,7.766301,7.979189,7.853582,8.170834
MIR1302-2HG,7.560175,7.560175,7.560175,7.560175,7.644686,7.560175,7.653864,7.560175,7.649028,7.560175,...,7.690715,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.643885
MIR1302-2,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,...,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175


In [4]:
#subset_df_drop = subset_df_drop.multiply(100)

##trying to un-log normalize data
test = 2**(subset_df_drop)

In [5]:
test.head()

,3081_BL,3447_BL,3232_BL,3764_BL,3810_BL,3201_BL,3057_V06,4117_BL,3174_V04,3661_V04,...,41283_V02,50746_V02,41282_V02,41375_V02,41292_V04,41356_V02,41296_V02,41294_V04,41314_V02,42308_V02
ID,,,,,,,,,,,,,,,,,,,,,
DDX11L1,207.297315,233.761347,247.117185,214.358662,227.083698,205.804813,188.729310,188.729310,224.599756,188.729310,...,201.201370,229.966596,221.771895,203.111931,211.225448,216.429142,208.801765,223.271948,224.142233,244.475744
WASH7P,456.623643,344.572225,410.185202,383.711388,413.820844,436.898264,407.992854,345.518469,438.787276,478.201917,...,450.739896,453.357213,395.768604,329.422450,379.417344,501.995802,348.609580,349.775096,411.110779,433.146288
MIR6859-1,237.402110,227.174853,220.578007,232.297968,245.078794,229.006041,206.879580,210.475332,188.729310,223.071577,...,305.835772,249.470340,227.361429,235.156223,259.216682,260.453883,217.715639,252.333631,231.293639,288.181439
MIR1302-2HG,188.729310,188.729310,188.729310,188.729310,200.115068,188.729310,201.392133,188.729310,200.718244,188.729310,...,206.602647,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,200.003971
MIR1302-2,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,...,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310


In [9]:
subset_df_drop.head()

,3081_BL,3447_BL,3232_BL,3764_BL,3810_BL,3201_BL,3057_V06,4117_BL,3174_V04,3661_V04,...,41283_V02,50746_V02,41282_V02,41375_V02,41292_V04,41356_V02,41296_V02,41294_V04,41314_V02,42308_V02
ID,,,,,,,,,,,,,,,,,,,,,
DDX11L1,769.555762,786.889259,794.905153,774.388290,782.708033,768.513291,756.017468,756.017468,781.121255,756.017468,...,765.249632,784.528051,779.293274,766.613117,772.263985,775.775096,770.599010,780.265819,780.827070,793.354752
WASH7P,883.486175,842.866261,868.013163,858.387777,869.286250,877.115356,867.240007,843.261902,877.737788,890.147610,...,881.615134,882.450443,862.851336,836.379507,856.764182,897.153149,844.546841,845.028376,868.338339,875.871054
MIR6859-1,789.118895,782.765934,778.514514,785.983272,793.710184,783.924185,769.264744,771.750734,756.017468,780.136289,...,825.661335,796.272449,782.884372,787.747570,801.801476,802.488413,776.630124,797.918869,785.358178,817.083361
MIR1302-2HG,756.017468,756.017468,756.017468,756.017468,764.468599,756.017468,765.386352,756.017468,764.902794,756.017468,...,769.071493,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,764.388484
MIR1302-2,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,...,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468


In [6]:
nodrop = (subset_df_nodrop.index).tolist()
drop = (subset_df_drop.index).tolist()
print(len(nodrop))
print(len(drop))

58780
57705


In [7]:
scaden_genes = pd.read_csv("{}scaden_training_genes.csv".format(WRKDIR))
scaden_genes = scaden_genes['gene'].tolist()

In [8]:
overlap_nodrop = list(set(nodrop) & set(scaden_genes))
overlap_drop = list(set(drop) & set(scaden_genes))
print(len(overlap_nodrop))
print(len(overlap_drop))

10509
10501


In [11]:
with open ("{}ppmi_genes.txt".format(WRKDIR), 'w') as f:
    for item in drop:
        f.write("%s\n" % item)

In [10]:
with open ("{}scaden_ppmi_overlap_genes.txt".format(WRKDIR), 'w') as f:
    for item in overlap_drop:
        f.write("%s\n" % item)


In [5]:
subset_df_drop.head(n=10)

,3081_BL,3447_BL,3232_BL,3764_BL,3810_BL,3201_BL,3057_V06,4117_BL,3174_V04,3661_V04,...,41283_V02,50746_V02,41282_V02,41375_V02,41292_V04,41356_V02,41296_V02,41294_V04,41314_V02,42308_V02
ID,,,,,,,,,,,,,,,,,,,,,
DDX11L1,769.555762,786.889259,794.905153,774.388290,782.708033,768.513291,756.017468,756.017468,781.121255,756.017468,...,765.249632,784.528051,779.293274,766.613117,772.263985,775.775096,770.599010,780.265819,780.827070,793.354752
WASH7P,883.486175,842.866261,868.013163,858.387777,869.286250,877.115356,867.240007,843.261902,877.737788,890.147610,...,881.615134,882.450443,862.851336,836.379507,856.764182,897.153149,844.546841,845.028376,868.338339,875.871054
MIR6859-1,789.118895,782.765934,778.514514,785.983272,793.710184,783.924185,769.264744,771.750734,756.017468,780.136289,...,825.661335,796.272449,782.884372,787.747570,801.801476,802.488413,776.630124,797.918869,785.358178,817.083361
MIR1302-2HG,756.017468,756.017468,756.017468,756.017468,764.468599,756.017468,765.386352,756.017468,764.902794,756.017468,...,769.071493,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,764.388484
MIR1302-2,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,...,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468,756.017468


In [ ]:
##check for missing entries


In [7]:
test.index.names = ['']
#making row index into first column
test.reset_index(level=0, inplace=True)

In [6]:
subset_df_drop.index.names = ['']
#making row index into first column
subset_df_drop.reset_index(level=0, inplace=True)

##making column headers into first row
#headers = subset_df_drop.columns
#df.rename(columns = header)
#subset_df_drop.loc[0] = headers


In [8]:
test.head()

,,3081_BL,3447_BL,3232_BL,3764_BL,3810_BL,3201_BL,3057_V06,4117_BL,3174_V04,...,41283_V02,50746_V02,41282_V02,41375_V02,41292_V04,41356_V02,41296_V02,41294_V04,41314_V02,42308_V02
0,DDX11L1,207.297315,233.761347,247.117185,214.358662,227.083698,205.804813,188.729310,188.729310,224.599756,...,201.201370,229.966596,221.771895,203.111931,211.225448,216.429142,208.801765,223.271948,224.142233,244.475744
1,WASH7P,456.623643,344.572225,410.185202,383.711388,413.820844,436.898264,407.992854,345.518469,438.787276,...,450.739896,453.357213,395.768604,329.422450,379.417344,501.995802,348.609580,349.775096,411.110779,433.146288
2,MIR6859-1,237.402110,227.174853,220.578007,232.297968,245.078794,229.006041,206.879580,210.475332,188.729310,...,305.835772,249.470340,227.361429,235.156223,259.216682,260.453883,217.715639,252.333631,231.293639,288.181439
3,MIR1302-2HG,188.729310,188.729310,188.729310,188.729310,200.115068,188.729310,201.392133,188.729310,200.718244,...,206.602647,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,200.003971
4,MIR1302-2,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,...,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310,188.729310


In [ ]:
subset_df_drop.head(n=10)

In [12]:
#subset_df_drop.to_csv('{}transpose.x100.genes.ppmi.v121018.vst.quants.txt'.format(WRKDIR), sep='\t', index=False, header=False)
subset_df_drop.to_csv('{}transpose.x100.genes.ppmi.v121018.vst.quants.txt'.format(WRKDIR), sep='\t', index=False)


In [ ]:
quants_df.to_csv('{}genes.ppmi.v121018.vst.quants.csv'.format(WRKDIR))

In [9]:
test.to_csv('{}2^x.genes.ppmi.v121018.vst.quants.txt'.format(WRKDIR), sep='\t', index=False)


In [16]:
##trying with another dataset

In [4]:
#load expression data
quants_df = pd.read_csv('{}vsdCountsPPMI_fullB38.tsv'.format(WRKDIR), sep='\t')
#load the gencode transcript annotations data
gencode = pd.read_csv('{}gencode.v29.transcripts.txt.gz'.format(WRKDIR), sep='\t', comment='#')
#create a dictionary of gene IDs (current feature/column labels) to gene names map
id_dict = dict(zip(gencode['gene_id'], gencode['gene_name'])) 


In [16]:
quants_df.head(n=20)

ID,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,MIR1302-2,FAM138A,OR4G4P,OR4G11P,OR4F5,AL627309.1,...,RF01518,RNA5-8SN2,RF01518,RNA5-8SN2,AC007325.3,AC007325.1,AC007325.4,AC007325.2,RF00026,RF00003
3081_BL,7.695558,8.834862,7.891189,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.997352,...,7.560175,7.560175,8.197396,7.560175,7.560175,7.560175,7.846998,7.655922,7.560175,7.560175
3447_BL,7.868893,8.428663,7.827659,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,8.261947,...,7.560175,7.669505,7.967991,7.714754,7.560175,7.669505,7.982205,7.560175,7.560175,7.560175
3232_BL,7.949052,8.680132,7.785145,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,8.035737,...,7.560175,7.560175,7.994626,7.560175,7.560175,7.560175,7.672745,7.672745,7.560175,7.560175
3764_BL,7.743883,8.583878,7.859833,7.560175,7.560175,7.560175,7.710206,7.560175,7.560175,8.233643,...,7.710206,7.710206,8.458600,7.710206,7.560175,7.666286,7.772255,7.666286,7.560175,7.560175
3810_BL,7.827080,8.692863,7.937102,7.644686,7.560175,7.560175,7.560175,7.560175,7.560175,8.123588,...,7.560175,7.560175,7.840068,7.560175,7.560175,7.560175,7.840068,7.560175,7.560175,7.560175
3201_BL,7.685133,8.771154,7.839242,7.560175,7.560175,7.560175,7.560175,7.648547,7.560175,8.533732,...,7.560175,7.560175,7.685133,7.560175,7.560175,7.560175,7.713193,7.560175,7.560175,7.560175
3057_V06,7.560175,8.672400,7.692647,7.653864,7.560175,7.653864,7.560175,7.653864,7.560175,8.609404,...,7.560175,7.722392,7.824842,7.692647,7.560175,7.653864,7.922144,7.722392,7.560175,7.560175
4117_BL,7.560175,8.432619,7.717507,7.560175,7.560175,7.651041,7.560175,7.651041,7.560175,8.622772,...,7.560175,7.560175,7.763223,7.560175,7.560175,7.560175,7.874375,7.651041,7.560175,7.560175
3174_V04,7.811213,8.777378,7.560175,7.649028,7.560175,7.560175,7.560175,7.560175,7.560175,8.607152,...,7.560175,7.649028,7.714024,7.560175,7.560175,7.560175,7.879935,7.560175,7.560175,7.560175
3661_V04,7.560175,8.901476,7.801363,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,8.248689,...,7.651426,7.560175,7.848306,7.560175,7.560175,7.651426,7.833566,7.560175,7.560175,7.560175


In [ ]:
#rename the features/columns using the dict (mapping)
#quants_df.rename(columns=id_dict, inplace=True)
quants_df.replace(id_dict, inplace = True)


In [12]:
print('nohup python replace_gene_names_vsdCounts.py > /home/mooreank/deconvolution/replace_gene_names_vsdCounts.log')

nohup python replace_gene_names_vsdCounts.py > /home/mooreank/deconvolution/replace_gene_names_vsdCounts.log


In [24]:
quants_df.head()

,Ensembl_ID,5104-SL-0095,5104-SL-0287,5104-SL-0479,5104-SL-0671,5104-SL-0767,5104-SL-0863,5104-SL-1055,5104-SL-1151,5104-SL-1247,...,5104-SL-2868,5104-SL-4064,5104-SL-4052,5104-SL-3648,5104-SL-0748,5104-SL-0400,5104-SL-0358,5104-SL-2933,5104-SL-3855,5104-SL-3127
0,ENSG00000223972.5,7.727660,7.817858,7.752089,7.794147,7.725227,7.830713,7.774505,7.808297,7.771638,...,7.761120,7.697284,7.656655,7.745397,7.560175,7.974044,7.947337,7.859412,8.931374,7.781492
1,ENSG00000227232.5,9.082737,8.706556,8.765745,8.622111,8.679957,8.719980,8.629412,8.479587,8.523566,...,8.551131,8.432209,8.451177,8.576544,10.210801,8.497224,8.666598,8.366279,9.488363,8.451255
2,ENSG00000278267.1,7.824770,7.759881,7.813914,7.941557,7.829459,7.830713,7.863006,7.808297,7.819047,...,8.008064,7.783933,7.832706,7.862297,9.920318,7.954906,7.928544,7.713883,8.419082,7.880549
3,ENSG00000243485.5,7.560175,7.723278,7.695929,7.695357,7.725227,7.670756,7.711798,7.560175,7.560175,...,7.560175,7.560175,7.560175,7.560175,7.560175,7.737144,7.644902,7.560175,8.658735,7.560175
4,ENSG00000284332.1,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,...,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175,7.560175


In [19]:
list(quants_df.columns.values)
#len(ppmi_genes)
scaden_genes = pd.read_csv('{}/scaden_training_genes.csv'.format(WRKDIR))
scaden_genes = scaden_genes['gene'].tolist()
overlap = set(ppmi_genes) & set(scaden_genes)
len(overlap)

In [7]:
#load expression data
quants_df = pd.read_csv('{}ppmi.v121018.transcripts.csv.gz'.format(WRKDIR), sep='\t')
#load the gencode transcript annotations data
gencode = pd.read_csv('{}gencode.v29.transcripts.txt.gz'.format(WRKDIR), sep='\t', comment='#')

pd.set_option('display.max_columns', None)


In [8]:
quants_df.head()                 

,Name,1009_POOLP1,1009_POOLP1.1,1009_POOLP1.2,1009_POOLP1.3,1009_POOLP1.4,1009_POOLP1.5,1009_POOLP1.6,1009_POOLP1.7,1009_POOLP1.8,...,90456_V06,90456_V08,91097_V02,91097_V04,91097_V06,92490_V02,92490_V04,92490_V06,92834_V02,92834_V04
0,ENST00000387460.2,0.00000,1.48536,0.00000,6.24678,0.946416,1.44405,1.49447,9.22938,6.26328,...,1.68697,3.13616,9.32977,3.93891,10.13110,5.1449,4.66744,0.00000,0.000000,22.30780
1,ENST00000387459.1,3.67079,37.13410,4.07591,31.23390,13.249800,40.43350,20.92260,66.14390,39.95970,...,8.43484,4.70425,7.46381,2.62594,6.07868,142.7710,12.44650,5.23745,0.548268,4.87984
2,ENST00000361681.2,5.52179,13.59970,9.06891,17.43810,7.017470,18.18490,15.25670,21.38380,14.30350,...,6.55639,5.72548,2.85848,4.09105,4.57808,88.4624,12.56880,3.59653,3.212510,4.56904
3,ENST00000361567.2,2.08765,2.79203,5.40794,8.15942,4.277520,8.39407,5.11948,6.32331,4.65510,...,7.11264,8.53137,10.67530,7.79557,6.14218,10.4539,12.37070,8.29453,7.903150,6.96980
4,ENST00000387449.1,1.08914,47.26150,0.00000,9.99484,6.115300,1.57533,10.46130,15.51050,8.76859,...,1.54639,4.70425,11.19570,6.56485,11.05210,11.5760,3.11163,3.49163,1.762290,1.81251


In [9]:
gencode.head()

,seqid,source,type,start,end,score,strand,phase,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,chr1,HAVANA,transcript,11869,14409,NaN,+,NaN,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1.0,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,12010,13670,NaN,+,NaN,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,transcribed_unprocessed_pseudogene,DDX11L1-201,NaN,basic,OTTHUMT00000002844.2,NaN,NaN,PGO:0000019,NaN,NaN
2,chr1,HAVANA,transcript,14404,29570,NaN,-,NaN,ENSG00000227232.5,unprocessed_pseudogene,...,unprocessed_pseudogene,WASH7P-201,NaN,basic,OTTHUMT00000002839.1,NaN,NaN,PGO:0000005,NaN,NaN
3,chr1,ENSEMBL,transcript,17369,17436,NaN,-,NaN,ENSG00000278267.1,miRNA,...,miRNA,MIR6859-1-201,NaN,basic,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,transcript,29554,31097,NaN,+,NaN,ENSG00000243485.5,lincRNA,...,lincRNA,MIR1302-2HG-202,5.0,basic,OTTHUMT00000002840.1,NaN,NaN,NaN,NaN,NaN


In [ ]:
#create a dictionary of gene IDs (current feature/column labels) to gene names map
id_dict = dict(zip(gencode['transcript_id'], gencode['gene_name'])) 


#rename the features/columns using the dict (mapping)
#quants_df.rename(columns=id_dict, inplace=True)
quants_df.replace(id_dict, inplace = True)


In [13]:
print('nohup python replace_gene_names_ppmi.py > /home/mooreank/deconvolution/replace_gene_names_ppmi.log')

nohup python replace_gene_names_ppmi.py > /home/mooreank/deconvolution/replace_gene_names_ppmi.log


In [ ]:
quants_df.head()

In [ ]:
list(quants_df.columns.values)
#len(ppmi_genes)
scaden_genes = pd.read_csv('{}/scaden_training_genes.csv'.format(WRKDIR))
scaden_genes = scaden_genes['gene'].tolist()
overlap = set(ppmi_genes) & set(scaden_genes)
len(overlap)